# LLM Router Chain Test
#### *LLM 체인 라우팅 적용하기*

<br><br><hr>

## 00. 기본 설정

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lgdx_team2_routerchain")

LangSmith 추적을 시작합니다.
[프로젝트명]
lgdx_team2_routerchain


In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import json
import re
from operator import itemgetter

# 랭체인 환경 설정
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

# VectorDB - FAISS
from langchain_community.vectorstores import FAISS


<br><br><hr>

## 01. 벡터DB 불러오기

In [4]:
### 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\USER\AppData\Local\Temp\ipykernel_5196\3142432257.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')
c:\workspaces\LGDXteam2\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# 벡터스토어 로드
new_vector_store = FAISS.load_local("../movie_4000_vectorstore_faiss",
                                    embeddings=embeddings,
                                    allow_dangerous_deserialization=True)

<br><br><hr>

## 02. Router Chain: 사용자 질문 유형 구분

- 정보검색
- 추천요청
- 일반대화 (`default_chain`)

In [14]:
# 1. 사용자 입력값 유형 분류용 프롬프트
classification_template = """사용자의 입력을 다음 세 가지 범주 중 하나로 분류하세요:
1️⃣ **"정보검색"**: 특정 영화, 드라마, 배우, 감독, 러닝타임, 개봉 연도, 수상 내역, 필모그래피 등 **사실적인 정보를 찾는 질문**
   - 기대되는 응답 예시: 배우가 출연한 드라마/영화 목록, 특정 연도의 개봉작 리스트 등
2️⃣ **"추천요청"**: 특정 장르, 배우, 테마(예: 좀비, 시간여행), 감성(예: 힐링, 긴장감) 등에 대한 **추천을 요청하는 질문**
   - 기대되는 응답 예시: 특정 조건을 만족하는 영화/드라마 추천
3️⃣ **"일반대화"**: 서비스와 무관한 일반적인 대화 (예: 날씨, AI 관련 질문, 잡담)

#### **예시 형식**
{{
  "type": "정보검색"
}}

<user_input>
{user_input}
</user_input>
"""
classification_prompt = ChatPromptTemplate.from_template(classification_template)

In [15]:
# 2. LLM을 이용한 질문 유형 분류 체인
classification_chain = (
  classification_prompt
  | ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=GEMINI_API_KEY)
  | StrOutputParser()
)

In [16]:
# 질문 분류 테스트
# 정보 검색 예상 질문
print(classification_chain.invoke({'user_input': "2023년에 개봉한 액션 영화 뭐 있어?"}))

# 추천 요청 예상 질문
print(classification_chain.invoke({'user_input': '디카프리오가 주연한 영화 추천해줘.'}))

# 일반 대화 예상 질문
print(classification_chain.invoke({'user_input': "너가 제일 좋아하는 영화 뭐야?"}))

```json
{
  "type": "정보검색"
}
```

```json
{
  "type": "추천요청"
}
```

```json
{
  "type": "일반대화"
}
```



In [26]:
# JSON 결과를 올바르게 파싱하는 함수
def preprocess_classification_result(user_input: str):
    classification_result = classification_chain.invoke({"user_input": user_input})

    # JSON 문자열에서 ```json``` 제거 (정규식 활용)
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', classification_result, re.DOTALL)
    if json_match:
        clean_json_str = json_match.group(1)                       # 중괄호 {} 내부만 추출
    else:
        clean_json_str = classification_result                     # ```json``` 태그가 없을 경우 그대로 사용

    # JSON 문자열을 dict 자료형으로 변환
    try:
        classification_data = json.loads(clean_json_str)
        type_value = classification_data.get("type", "일반대화")   # 기본값을 '일반대화'로 설정
        # keywords = classification_data.get("keywords", [])         # 기본값을 빈 리스트로 설정

    except json.JSONDecodeError as e:
        print(f"⚠️ JSONDecodeError: {e}")  # JSON 변환 오류 확인
        type_value = "일반대화"           # JSON 변환 오류 시 기본값 설정
        # keywords = []
    
    # return {"type": type_value, "keywords": keywords}
    return {"type": type_value}

In [27]:
# 딕셔너리 자료형을 string으로 변환하는 함수
def format_change(classification_result: dict) -> str:
    type_value = classification_result.get("classification_result", {}).get("type", "일반대화")
    # keywords = classification_result.get("classification_result", {}).get("keywords", [])
    user_input = classification_result.get("user_input", "")

    # string 자료형으로 변경
    # formatted_str = f"type: '{type_value}', keywords: {keywords}, user_input: '{user_input}'"
    formatted_str = f"type: '{type_value}', user_input: '{user_input}'"
    return formatted_str

<br><br><hr>

## 03. Destination Chain

#### *1. default-chain*

In [28]:
# default_chain 생성 (사용자의 의미없는 입력값에 대해 정해진 답변을 할 때)
# 프롬프트 템플릿 설정
default_template = """
"You are a chatbot that must always respond with '🐶: 멍멍!'.
No matter what question the user asks, always reply with '🐶: 멍멍!'"

[사용자 입력과 분류 결과]:
{classification_result}
"""
default_prompt = ChatPromptTemplate.from_template(default_template)

# Google Gemini 모델 생성
def load_gemini():
    model = ChatGoogleGenerativeAI(
        model='gemini-1.5-flash',
        temperature=0,
        max_tokens=1000,
        api_key=GEMINI_API_KEY
    )
    print(">>>>>>> model loaded...")
    return model

default_llm = load_gemini()

# langchain 체인 구성
default_chain = (
  {"classification_result": RunnablePassthrough()}
  | default_prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | default_llm            # llm이 원하는 답변을 만듦
  # | StrOutputParser()
)

>>>>>>> model loaded...


#### *2. search-chain*

<br><hr>

#### *3. recommendation-chain*
- 추천 목록을 반환하는 체인
- `next_input`이 필요 없음 + 정해진 자료형으로 답해야 함  
  => `RouterOutputParser`를 사용해보는 게 좋을 거 같음..~
- 이 체인 뒤에 사용자 시청 기록 기반으로 반환된 추천 목록에서 5개를 정하는 작업을 해야 함

<br>

- 장르 기반 추천 ⇒ 장르
- 줄거리(키워드/컨셉) 기반 추천 ⇒ 줄거리
- 특정 연도 및 시대별 콘텐츠 추천 ⇒ 줄거리에 언급되는 시대배경/연도
==================================================
- 콘텐츠 정보 기반 추천 ⇒ 어떤 행이 있는지 봐야할듯
- 인기 있는 콘텐츠 추천 ⇒ 시청횟수가 많은 것
- 리뷰(감성) 기반 추천 ⇒ 리뷰
- 사용자 선호 기반 추천 ⇒ 사용자 시청기록


In [29]:
# 검색기 생성
retriever = new_vector_store.as_retriever(
    search_type="mmr",   
    search_kwargs={"k": 20,              # 반환할 문서 수 (default: 4)
                   "fetch_k": 50,       # MMR 알고리즘에 전달할 문서 수
                   "lambda_mult": 0.5,    # 결과 다양성 조절 (default: 0.5),
                   }
)

# 프롬프트 템플릿 설정
template = """
You are a movie-recommendation chatbot.
You must only answer based on the given context.
Do not generate answers that are not directly supported by the context.
아래 JSON 리스트 형식으로 반환하세요.  
JSON 이외의 설명은 하지 마세요.  
추천 순서는 `id`로 표시하며, `genre`(장르)와 `title`(영화 제목)은 retrieved_context에서 가져온 메타데이터를 기반으로 합니다.

---
예제 출력 형식:
```json
{{
    {{"id": 1, "genre": "다큐멘터리", "title": "견자단의 용호무"}},
    {{"id": 2, "genre": "드라마", "title": "스프링 송"}},
    {{"id": 3, "genre": "드라마", "title": "디어 마이 프렌드"}}
}}

[사용자 입력과 사용자 입력값의 유형 및 키워드]:
{user_input}

[Context]:
{retrieved_context}

[Answer]:
"""
prompt = ChatPromptTemplate.from_template(template)

# Google Gemini 모델 생성
def load_gemini(system_instruction):
    model = ChatGoogleGenerativeAI(
        model='gemini-1.5-flash',
        temperature=0.3,
        max_tokens=5000,
        system_instruction=system_instruction,
        api_key=GEMINI_API_KEY
    )
    print(">>>>>>> model loaded...")
    return model

system_instruction = """you are a movie-recommendation chatbot. you must answer based on given data."""
llm = load_gemini(system_instruction)

# langchain 체인 구성
recommendation_chain = (
  {"user_input":RunnablePassthrough(),
    "retrieved_context": retriever,
  }
  # question(사용자의 질문) 기반으로 연관성이 높은 문서 retriever 수행 >> format_docs로 문서를 하나로 만듦
  | prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | llm                  # llm이 원하는 답변을 만듦
  | StrOutputParser()
)

>>>>>>> model loaded...


<br><br><hr>

## 04. Full Chain 연결

In [30]:
def process_user_input(classification_result: dict):
    # 사용자의 입력 유형 분류
    classification_data = classification_result.get("classification_result", {})  # 내부 딕셔너리 추출
    type_value = classification_data.get("type", "일반대화")  # 기본값 설정
    # keywords = classification_data.get("keywords", [])  # 기본값 설정
    
    print(f"===================== Type: {type_value}")
    # print(f"===================== Keywords: {keywords}")
    
    if type_value == '정보검색':
        return "정보검색 체인 실행은 여기!!!"
    elif type_value == '추천요청':
        formatted_string = format_change(classification_result)
        return recommendation_chain.invoke(formatted_string)
    else:
        return default_chain.invoke({"classification_result": classification_result})

In [31]:
full_chain = (
  {"classification_result": RunnableLambda(preprocess_classification_result),
   "user_input":itemgetter("user_input")}
  | RunnableLambda(process_user_input)
  | StrOutputParser()  
)

In [32]:
full_chain.invoke({"user_input": "미국 뉴욕을 배경으로 한 액션 영화를 추천해줘."})

===================== Type: 추천요청


'```json\n[\n  {\n    "id": "f09b0b34-df98-48df-a722-7837b5e71f33",\n    "genre": "액션/어드벤쳐",\n    "title": "액션히어로"\n  }\n]\n```\n'